# 04. Modeling (Feature Elimination)

---

In this notebook, I build Principal Component Regression models with 1) dataset from `02_Feature_Engineering`, which all engineered terms were added and 2) dataset from `03_Modeling_Feature_Elimination`, which unimportant features were eliminated from the first dataset by using L1 penalty from Lasso Regression.

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import functions as fc

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV, ElasticNetCV
from sklearn.decomposition import PCA
from sklearn import metrics

%matplotlib inline

## Dataset 1 (All Features)
This dataset was prepared in `02_Feature_Eningeering` notebook. This dataset has all engineered features in addition to all original features

In [2]:
# Read in train data

df = pd.read_csv('./datasets/clean_train_engineered_terms.csv')

In [3]:
df.shape

(2051, 25203)

In [4]:
X = df.drop(['Id', 'PID', 'SalePrice'], axis = 1)
y = df['SalePrice']

Perform train, test split and scale the dataset before Principal Component Analysis (PCA)

In [5]:
rs = 112

# Instantiate our StandardScaler.
ss = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = rs)

# Scale X_Train
X_train_sc = ss.fit_transform(X_train)

# Scale X_Test
X_test_sc = ss.transform(X_test)

In [6]:
# Instantiate PCA.
pca = PCA()
# Fit PCA
pca.fit(X_train_sc)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [7]:
# Transform PCA on the training data.
Z_train = pca.transform(X_train_sc)

Z_test = pca.transform(X_test_sc)

In [8]:
# Check out the resulting data.

pd.DataFrame(Z_train).head()

,0,1,2,3,4,5,6,7,8,9,...,1528,1529,1530,1531,1532,1533,1534,1535,1536,1537
0,23.646289,-2.618827,-13.767287,-17.963663,-1.964502,3.316205,0.217721,1.508359,-2.461785,1.137101,...,-0.003001,0.001278,0.000157,-0.000697,-0.000355,-0.000635,-0.001121,-1.273821e-06,4.621303e-14,-1.554312e-15
1,-19.030450,18.218628,-9.556384,-14.827636,1.244923,-6.928042,-17.559337,11.592378,-2.433828,-0.181161,...,-0.050907,0.047616,-0.018166,-0.010820,0.009490,0.000002,0.004909,-1.065034e-04,-3.469447e-14,2.331468e-15
2,-24.283959,10.808308,-4.811671,-7.049423,7.234242,-14.152412,-5.818647,7.230979,4.484563,-7.032775,...,-0.044067,0.066571,-0.083929,0.019417,-0.016341,-0.003363,0.015950,-2.893721e-04,-1.498801e-15,2.046974e-15
3,8.816447,-16.602130,0.876916,-0.406625,0.966570,15.729748,-10.978971,-13.124946,-2.604664,-1.942068,...,-0.002197,0.000178,0.000219,-0.001987,-0.002512,0.001649,-0.000133,-3.807228e-06,6.050715e-14,4.302114e-16
4,-17.075123,-15.775845,18.868727,25.696248,-3.533555,-4.921675,23.145130,36.373807,31.035715,-9.822331,...,-0.000121,-0.000134,-0.000015,0.000033,0.000034,-0.000056,-0.000012,6.165308e-08,-1.743050e-14,4.718448e-15


Check how much each principal components explains the variance.

In [9]:
# Pull the explained variance attribute.
var_exp = pca.explained_variance_ratio_
print(f'Explained variance (first 10 components): {np.round(var_exp[:10], 3)}') 

print('')

# Generate the cumulative explained variance.
cum_var_exp = np.cumsum(var_exp)
print(f'Cumulative explained variance (first 10 components): {np.round(cum_var_exp[:10], 3)}')

Explained variance (first 10 components): [0.04  0.021 0.021 0.017 0.014 0.013 0.011 0.01  0.009 0.009]

Cumulative explained variance (first 10 components): [0.04  0.061 0.082 0.099 0.113 0.126 0.137 0.147 0.157 0.166]


### Principal Component Regression (All features)
From the PCA above, we saw first 10 components explains about 15 percent of the variance in the dataset. In this part, I examine what number of the component yields the best score.

In [10]:
# Instantiate PCA 
pca = PCA()

# Fit PCA to training data.
pca.fit(X_train_sc)

# Instantiate linear regression model.
lm = LinearRegression()

# Transform Z_train and Z_test.
Z_train = pca.transform(X_train_sc)
Z_test = pca.transform(X_test_sc)

# Fit on Z_train.
lm.fit(Z_train, y_train)

# Score on training and testing sets.
print(f'Training Score: {round(lm.score(Z_train, y_train),4)}')
print(f'Testing Score: {round(lm.score(Z_test, y_test),4)}')

Training Score: 1.0
Testing Score: -1.3694060669428015e+22


When all principal components are included in the Linear Regression, the test score is negative. This indicates that the model performs very poorly and baseline model, which predicts every sale price to be the average, yields better prediction score.           
Below, I attempt to find the number of principal components that returns the best prediction score (R2). In order to find score that has the best bias and variance trade off, I search for the number of principal components that returns the minimal difference between its train score and test score.

In [30]:
n_components_to_try = list(range(1, len(cum_var_exp),5))
score_dict = {}

for n_comp in n_components_to_try:
    pca = PCA(n_components = n_comp)
    # Fit PCA to training d2ata.
    pca.fit(X_train_sc)

    # Instantiate linear regression model.
    lm = LinearRegression()

    # Transform Z_train and Z_test.
    Z_train = pca.transform(X_train_sc)
    Z_test = pca.transform(X_test_sc)
    
    # Fit on Z_train.
    lm.fit(Z_train, y_train)
    
    score_dict[n_comp] = (round(lm.score(Z_train, y_train),4), round(lm.score(Z_test, y_test),4))

In [31]:
best_diff = 1000
best_score = 0
for key in score_dict:
    train = score_dict[key][0]
    test = score_dict[key][1]
    diff = abs(train - test)
    if diff < best_diff:
        best_diff = diff
        best_score = key

print(f"Best n_component is {best_score} \n,which has train and test score of {score_dict[best_score]}")

Best n_component is 136 
,which has train and test score of (0.9559, 0.8994)


## Dataset 2 
This dataset was prepared in notebook `03_Modeling_Feature_Elimination`. From the first dataset, which included all engineered features, unimportant features were eliminated using L1 penalty (Lasso regression)

In [21]:
df_n = pd.read_csv("./datasets/lasso_selected_features.csv")

In [22]:
X = df_n
y = df['SalePrice']

In [23]:
rs = 112

# Instantiate our StandardScaler.
ss = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = rs)

# Scale X_Train
X_train_sc = ss.fit_transform(X_train)

# Scale X_Test
X_test_sc = ss.transform(X_test)

In [24]:
# Instantiate PCA.
pca = PCA()
# Fit PCA
pca.fit(X_train_sc)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [25]:
# Transform PCA on the training data.
Z_train = pca.transform(X_train_sc)

Z_test = pca.transform(X_test_sc)

In [26]:
# Check out the resulting data.

pd.DataFrame(Z_train).head()

,0,1,2,3,4,5,6,7,8,9,...,228,229,230,231,232,233,234,235,236,237
0,-5.076489,-0.215177,-0.672125,1.927722,-3.669929,0.672852,0.650356,0.517998,-0.641761,0.748424,...,-2.259842e-17,6.825424e-19,1.739888e-18,9.969115e-18,-2.413578e-18,1.070734e-17,-1.794527e-17,4.190153e-18,6.256088e-18,-1.055202e-16
1,0.604858,-1.970915,3.365762,0.446520,2.030953,-3.143049,-0.537811,-0.277820,0.251185,1.365585,...,7.769028e-17,2.285327e-19,-5.280321e-18,2.867591e-18,1.324637e-17,-1.661455e-17,6.879091e-17,-3.744321e-17,2.786641e-18,-2.581759e-16
2,0.780148,-0.471615,1.524440,-0.308748,1.858811,-1.001132,-0.666676,0.563845,0.027066,-0.700446,...,1.220342e-16,-6.369367e-20,-2.678236e-18,1.241288e-18,-2.318711e-18,6.242634e-19,3.756589e-17,-1.662653e-17,1.666443e-17,-3.414426e-16
3,-2.456971,-0.049042,-1.769326,1.630013,0.276528,-1.169301,0.022713,0.305967,-0.776484,2.007972,...,2.217913e-17,1.248360e-18,-9.861076e-18,8.076070e-19,-1.237897e-18,-7.398833e-18,4.450478e-17,-8.948492e-17,-3.537728e-17,-2.980745e-16
4,2.965629,6.999733,-7.626371,-1.509651,1.112076,-3.411387,-1.136671,-3.350284,4.223536,-9.395278,...,-9.755383e-16,5.178213e-18,-5.608896e-18,6.458799e-18,1.590171e-17,5.991922e-18,-3.189289e-17,1.349032e-16,-4.615504e-18,-2.279294e-16


In [27]:
# Pull the explained variance attribute.
var_exp = pca.explained_variance_ratio_
print(f'Explained variance (first 10 components): {np.round(var_exp[:10], 3)}') 
# if we use round without np., we can't use that on an array

print('')

# Generate the cumulative explained variance.
cum_var_exp = np.cumsum(var_exp)
print(f'Cumulative explained variance (first 10 components): {np.round(cum_var_exp[:10], 3)}')

Explained variance (first 10 components): [0.13  0.039 0.032 0.029 0.026 0.026 0.024 0.024 0.022 0.02 ]

Cumulative explained variance (first 10 components): [0.13  0.17  0.202 0.231 0.257 0.283 0.307 0.331 0.352 0.373]


### Principal Component Regression (Features selected from Lasso Regression)¶
From the PCA above, we saw first 10 components explains about 37 percent of the variance in the dataset 2. In this part, I examine what number of the component yields the best score.

In [28]:
n_components_to_try = list(range(1, len(cum_var_exp),1))
score_dict = {}

for n_comp in n_components_to_try:
    pca = PCA(n_components = n_comp)
    # Fit PCA to training data.
    pca.fit(X_train_sc)

    # Instantiate linear regression model.
    lm = LinearRegression()

    # Transform Z_train and Z_test.
    Z_train = pca.transform(X_train_sc)
    Z_test = pca.transform(X_test_sc)
    
    # Fit on Z_train.
    lm.fit(Z_train, y_train)
    
    score_dict[n_comp] = (round(lm.score(Z_train, y_train),4), round(lm.score(Z_test, y_test),4))

In [29]:
best_diff = 100
best_score = 0
for key in score_dict:
    train = score_dict[key][0]
    test = score_dict[key][1]
    diff = abs(train - test)
    if diff < best_diff:
        best_diff = diff
        best_score = key

print(f"Best n_component is {best_score} \n,which has train and test score of {score_dict[best_score]}")

Best n_component is 135 
,which has train and test score of (0.9561, 0.8959)


Principal Component Regression on the two datasets performed the same (scores were rounded up to two decimal points). Both scores however were not as high as the best score from the original dataset.

|                              | Dataset 1 | Dataset 2 |  
|------------------------------|-----------|-----------|
| Optimal Number of Components | 136       | 135       |
| Train Score (R2)             | 0.96      | 0.96      |
| Test Score (R2)              | 0.90      | 0.90      |